In [20]:
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython

import cv2
import numpy as np
import pandas as pd
import pandas as pd
import pickle
import glob
import os

from matplotlib.pyplot import *
from utility import *

from scipy import optimize, sqrt
from matplotlib.patches import Ellipse
import plotly.express as px

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
info = """v05_T2_R_3d,7302.47,228.58,7500
v06_T1_L_3d,7500,274.21,7500
v04_T1_L_3d,7420.48,308.71,7500
v02_T2_L_3d,7311.48,306.65,7500
v04_T1_R_3d,7326.07,319.37,7500
v01_T2_L_3d,7500,253.19,7500
v02_T1_R_3d,7411.04,281.72,7500
v03_T1_L_3d,7201.62,306.67,7500
v04_T2_R_3d,7424.77,214.81,7500
v05_T1_L_3d,7500,324.97,7500
v06_T1_R_3d,7476.69,397.99,7500
v03_T1_R_3d,7199.05,267.50,7500
v05_T2_L_3d,7403.31,405.86,7500
v02_T2_R_3d,7408.89,287.19,7500
v01_T2_R_3d,7500,340.53,7500
v04_T2_L_3d,7425.20,274.28,7500
v02_T1_L_3d,7212.35,253.20,7500
v01_T1_L_3d,7408.89,359.44,7500
v05_T1_R_3d,7500,317.93,7500
v06_T2_L_3d,7376.71,269.32,7500
v03_T2_L_3d,7257.84,243.52,7500
v03_T2_R_3d,7312.77,268.11,7500
v01_T1_R_3d,7206.34,305.86,7500"""
ImageInfo = collections.namedtuple('ImageInfo', 'max_height, max_peak, max_width')
image_info = {name: ImageInfo(*(float(x) for x in rest)) 
      for name, *rest in [x.split(',') for x in info.split('\n')]}

In [3]:
from sklearn.linear_model import LinearRegression
def adjust_tilt(img):
    median = np.median(img)
    q1 = np.percentile(img, 25)
    q3 = np.percentile(img, 75)
    iqr = q3 - q1
    poinst = np.where((img > median - 2 * iqr) & (img < median + 2 * iqr))
    y = img[poinst]
    X = np.array(poinst).T
    model = LinearRegression(n_jobs=8, normalize=True)
    model.fit(X, y)
    poinst = np.where(img > -10)
    y = img[poinst]
    X = np.array(poinst).T   
    y_pred = model.predict(X)
    print(model.coef_)
    adjusted = (y + y.mean() - y_pred).reshape(img.shape)
    negative_indices = np.where(img < 0)
    adjusted[negative_indices] = img[negative_indices]
    return adjusted

In [4]:
def rescale_image(img, img_info, scale=1):
    scales = np.array((img_info.max_height, img_info.max_width))/np.array(img.shape)
    print("scales = %f, %f"%tuple(scales))
    resized_img = cv2.resize(img, (int(img_info.max_width*scale), int(img_info.max_height*scale)))*scale
    return resized_img

In [5]:
def disect_path(path):
    base, ext = os.path.splitext(path)
    base = os.path.basename(base)
    directory = os.path.dirname(path)
    return directory, base, ext

In [6]:
def get_all_circles(rotated):
    allCircles = []
    for i, profile in enumerate(rotated):
        _kernel_size = int(np.percentile(get_cut_points(profile, profile.mean()), 25))
        #print("kernel size param %r"%_kernel_size)
        x, y, der1, der2, local_minima_2d, circles = extract_circles(np.arange(len(profile)), 
                                                             profile,
                                                             kernel_size_param=_kernel_size,
                                                             same_scale=True, verbose=False)       
        if i%300==0:
            draw_circles(x, y, circles)
            show()
        allCircles.append(circles)
    return allCircles
#plot(x, y)
#scatter(x[local_minima_2d], y[local_minima_2d])

In [18]:
columns = ['profile'] + list(Circle._fields)
def fun(image_name):
    print('Starting to process %s'%image_name)
    directory, base, ext = disect_path(image_name)
    base_name = os.path.splitext(image_name)[0]
    df_file_name =  base_name +'.df'
    if os.path.exists(df_file_name):
        print('dataframe %s already exists'%df_file_name)
        return
    with open(image_name, 'rb') as f:
        img = pickle.load(f)
    print("[*] image %s loaded"%base)
    try:
        img_info = image_info[base]
    except KeyError:
        print('image info not found')
        return
    resized_img = rescale_image(img, img_info, scale=1)
    print('[*] image resized')
    rotated, *_ = align_image(resized_img)
    print('[*] image rotated')
    aligned = adjust_tilt(rotated)
    print('[*] image adjusted for tilt')
    allCircles = get_all_circles(aligned)
    df = pd.DataFrame(((i, *circle) for i, circles in enumerate(allCircles) for circle in circles), columns=columns)
    print("[*] Dataframe to be saved as %s"%df_file_name)
    with open(df_file_name, 'wb') as f:
        pickle.dump(df, f)

In [19]:
from multiprocessing import Pool
pool = Pool(processes=24)
for image_name in glob.glob('dataset/*3d.np'):
    fun(image_name)
#pool.map(fun, list(glob.glob('dataset/*3d.np')))

Starting to process dataset/v02_T2_R_3d.np
dataframe dataset/v02_T2_R_3d.df already exists
Starting to process dataset/v01_T1_L_3d.np
dataframe dataset/v01_T1_L_3d.df already exists
Starting to process dataset/v01_T2_R_3d.np
dataframe dataset/v01_T2_R_3d.df already exists
Starting to process dataset/v04_T2_L_3d.np
dataframe dataset/v04_T2_L_3d.df already exists
Starting to process dataset/v04_T1_L_3d.np
dataframe dataset/v04_T1_L_3d.df already exists
Starting to process dataset/v04_T2_R_3d.np
dataframe dataset/v04_T2_R_3d.df already exists
Starting to process dataset/v03_T2_L_3d.np
dataframe dataset/v03_T2_L_3d.df already exists
Starting to process dataset/v06_T1_R_3d.np
dataframe dataset/v06_T1_R_3d.df already exists
Starting to process dataset/v05_T2_L_3d.np
dataframe dataset/v05_T2_L_3d.df already exists
Starting to process dataset/v06_T2_L_3d.np
dataframe dataset/v06_T2_L_3d.df already exists
Starting to process dataset/v02_T1_R_3d.np
dataframe dataset/v02_T1_R_3d.df already exists

/home/bzr0014/anaconda3/envs/tensorGpuNew/lib/python3.6/site-packages/scipy/optimize/minpack.py:436: RuntimeWarning:

Number of calls to function has reached maxfev = 800.



Skipped drawing circle at 4514 due to radius of curvature being too high
Skipped drawing circle at 1398 due to radius of curvature being too high
Skipped drawing circle at 5782 due to radius of curvature being too high
Skipped drawing circle at 7358 due to radius of curvature being too high
Skipped drawing circle at 1682 due to radius of curvature being too high
Skipped drawing circle at 3707 due to radius of curvature being too high
Skipped drawing circle at 3802 due to radius of curvature being too high
Skipped drawing circle at 2783 due to radius of curvature being too high
Skipped drawing circle at 4559 due to radius of curvature being too high
Skipped drawing circle at 4178 due to radius of curvature being too high
Skipped drawing circle at 4447 due to radius of curvature being too high
Skipped drawing circle at 5161 due to radius of curvature being too high
Skipped drawing circle at 6990 due to radius of curvature being too high
Skipped drawing circle at 675 due to radius of curv

/home/bzr0014/tmp/additive/utility.py:245: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



Skipped drawing circle at 1825 due to radius of curvature being too high
Skipped drawing circle at 550 due to radius of curvature being too high
Skipped drawing circle at 633 due to radius of curvature being too high
Skipped drawing circle at 6756 due to radius of curvature being too high
Skipped drawing circle at 5344 due to radius of curvature being too high
Skipped drawing circle at 6057 due to radius of curvature being too high
Skipped drawing circle at 975 due to radius of curvature being too high
Skipped drawing circle at 1991 due to radius of curvature being too high
Skipped drawing circle at 4393 due to radius of curvature being too high
Skipped drawing circle at 5044 due to radius of curvature being too high


IndexError: index 7488 is out of bounds for axis 1 with size 7488

In [21]:
with open('dataset/v06_T1_R_3d.df', 'rb') as f:
    df = pickle.load(f)

In [33]:
print(df.iloc[:, :-4].sample(10).round(decimals=2).to_latex())

\begin{tabular}{lrrrrrrrr}
\toprule
{} &  profile &   beg &   end &  index &       h &       cx &      cy &      r \\
\midrule
88198 &     5903 &  4664 &  4673 &   4668 &  211.38 &  4668.16 &  219.50 &   7.32 \\
43255 &     3508 &  3444 &  3457 &   3450 &  215.44 &  3444.39 &  235.02 &  14.67 \\
9517  &     1258 &  5246 &  5317 &   5281 &  201.56 &  5286.15 &  217.83 &  16.37 \\
70204 &     4675 &  5571 &  5582 &   5576 &  236.54 &  5585.30 &  230.28 &   8.21 \\
94882 &     6519 &  5714 &  5725 &   5719 &  207.36 &  5717.25 &  227.25 &   9.84 \\
89072 &     5975 &  6964 &  6973 &   6968 &  213.10 &  6973.88 &  202.81 &   3.89 \\
75594 &     4949 &  1504 &  1513 &   1508 &  213.70 &  1513.57 &  208.65 &   3.69 \\
14443 &     1781 &  3980 &  3987 &   3983 &  207.81 &  3987.41 &  213.81 &   6.27 \\
61332 &     4256 &  1248 &  1257 &   1252 &  234.75 &  1257.32 &  226.97 &   3.33 \\
81608 &     5299 &  6554 &  6561 &   6557 &  265.89 &  6539.61 &  216.73 &  20.69 \\
\bottomrule
\end{tabula

In [36]:
stats[0]

NameError: name 'stats' is not defined